#### Лабораторная работа № 2. Разработка нейросетевых функций. Операция Convolution 3D
#### БВТ2003 Суроп Максим Андреевич

In [8]:
import torch
import numpy as np
from typing import Union

In [9]:
class Conv3D:
    def __init__(self, in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple] = 1, groups: int = 1) -> None:
        # Инициализация параметров сверточного слоя
        self.in_channels, self.out_channels = in_channels, out_channels
        # Преобразование размера ядра, шага и групп в кортеж, если они заданы в виде целых чисел
        self.kernel_size, self.stride, self.groups = (kernel_size, kernel_size, kernel_size) if isinstance(kernel_size, int) else kernel_size, stride, groups

        # Инициализация весов (ядер свертки) и смещений
        self.weights = np.random.randn(out_channels, in_channels // groups, *self.kernel_size)
        self.bias = np.zeros((out_channels,))

    def forward(self, input_tensor: np.ndarray) -> np.ndarray:
        batch_size, in_channels, input_depth, input_height, input_width = input_tensor.shape
        kernel_size = self.kernel_size

        # Проверка корректности размера ядра относительно входного тензора
        if input_depth < kernel_size[0] or input_height < kernel_size[1] or input_width < kernel_size[2]:
            raise ValueError("Неверный размер ядра для входного тензора")

        # Извлечение размеров окна свертки и выходного тензора
        stride_d, stride_h, stride_w = (self.stride, self.stride, self.stride) if isinstance(self.stride, int) else self.stride
        output_depth = (input_depth - kernel_size[0]) // stride_d + 1
        output_height = (input_height - kernel_size[1]) // stride_h + 1
        output_width = (input_width - kernel_size[2]) // stride_w + 1

        # Инициализация результата свертки
        result = np.zeros((self.out_channels, output_depth, output_height, output_width))

        # Выполнение операции свертки
        for d in range(0, input_depth - kernel_size[0] + 1, stride_d):
            for i in range(0, input_height - kernel_size[1] + 1, stride_h):
                for j in range(0, input_width - kernel_size[2] + 1, stride_w):
                    # Извлечение окна свертки из входного тензора
                    window = input_tensor[:, :, d:d + kernel_size[0], i:i + kernel_size[1], j:j + kernel_size[2]]
                    # Вычисление свертки и добавление смещения
                    result[:, d // stride_d, i // stride_h, j // stride_w] = np.sum(window * self.weights, axis=(1, 2, 3, 4)) + self.bias

        return result


In [10]:
def run_3d_test(in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple],
                padding: Union[int, tuple], dilation: int, groups: int, bias: bool, padding_mode: str) -> None:
    # Создание экземпляра Conv3D и выполнение трехмерной свертки
    conv3d_layer = Conv3D(in_channels, out_channels, kernel_size, stride)
    result_3d = conv3d_layer.forward(np.random.randn(1, in_channels, 10, 10, 10))

    # Преобразование результатов в формат torch.Tensor для сравнения с PyTorch
    input_tensor_torch = torch.tensor(conv3d_layer.weights)
    output_tensor_torch = torch.nn.functional.conv3d(torch.tensor(conv3d_layer.weights), 
                                                      input_tensor_torch, 
                                                      bias=torch.tensor(conv3d_layer.bias), 
                                                      stride=conv3d_layer.stride, 
                                                      padding=padding,
                                                      dilation=dilation,
                                                      groups=1).numpy()

    # Проверка совпадения результатов
    are_results_equal = np.allclose(result_3d, output_tensor_torch)
    
    # Вывод информации о тесте
    print(f"Тест с параметрами:")
    print(f"  - in_channels: {in_channels}")
    print(f"  - out_channels: {out_channels}")
    print(f"  - kernel_size: {kernel_size}")
    print(f"  - stride: {stride}")
    print(f"  - padding: {padding}")
    print(f"  - dilation: {dilation}")
    print(f"  - groups: {groups}")
    print(f"  - bias: {bias}")
    print(f"  - padding_mode: {padding_mode}")
    print(f"\nСовпадение результатов: {are_results_equal}")
    print(f"\n{'-'*75}\n")


In [11]:
def test_3d_params() -> None:
    # параметры тестирования для первой группы
    run_3d_test(in_channels=8, out_channels=20, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
    
    # параметры тестирования для дополнительной группы
    run_3d_test(in_channels=3, out_channels=2, kernel_size=3, stride=(2, 2, 2), padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
    
    # параметры тестирования для дополнительной группы
    run_3d_test(in_channels=8, out_channels=20, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')

In [12]:
# параметры тестирования для дополнительной группы
input_tensor_3d = np.random.randn(1, 3, 5, 5, 5)
conv3d_layer = Conv3D(in_channels=3, out_channels=2, kernel_size=3, stride=(2, 2, 2))
output_tensor_3d = conv3d_layer.forward(input_tensor_3d)

print(f"Результат свертки (Conv3D):\n {output_tensor_3d}")

Результат свертки (Conv3D):
 [[[[ -6.60505856  -2.22292124]
   [ -4.16961661   0.99754294]]

  [[  7.98413282   8.399154  ]
   [ -9.11719122 -11.07604176]]]


 [[[ -5.452002     5.92787561]
   [ -0.27820987 -17.62846252]]

  [[-13.86865314  -9.56083168]
   [-11.31331035  -8.82855084]]]]


In [13]:
# преобразование результатов в формат torch.Tensor для сравнения с PyTorch
input_tensor_torch_3d = torch.tensor(input_tensor_3d)
output_tensor_torch_3d = torch.nn.functional.conv3d(input_tensor_torch_3d,
                                                     torch.tensor(conv3d_layer.weights),
                                                     bias=torch.tensor(conv3d_layer.bias),
                                                     stride=(2, 2, 2)).numpy()

print(f"Результат свертки (PyTorch):\n {output_tensor_torch_3d}")

Результат свертки (PyTorch):
 [[[[[ -6.60505856  -2.22292124]
    [ -4.16961661   0.99754294]]

   [[  7.98413282   8.399154  ]
    [ -9.11719122 -11.07604176]]]


  [[[ -5.452002     5.92787561]
    [ -0.27820987 -17.62846252]]

   [[-13.86865314  -9.56083168]
    [-11.31331035  -8.82855084]]]]]


In [14]:
test_3d_params()

Тест с параметрами:
  - in_channels: 8
  - out_channels: 20
  - kernel_size: 3
  - stride: 1
  - padding: 0
  - dilation: 1
  - groups: 2
  - bias: True
  - padding_mode: zeros

Совпадение результатов: False

---------------------------------------------------------------------------

Тест с параметрами:
  - in_channels: 3
  - out_channels: 2
  - kernel_size: 3
  - stride: (2, 2, 2)
  - padding: 0
  - dilation: 1
  - groups: 1
  - bias: True
  - padding_mode: zeros

Совпадение результатов: False

---------------------------------------------------------------------------

Тест с параметрами:
  - in_channels: 8
  - out_channels: 20
  - kernel_size: 3
  - stride: 1
  - padding: 0
  - dilation: 1
  - groups: 2
  - bias: True
  - padding_mode: zeros

Совпадение результатов: False

---------------------------------------------------------------------------

